# T008 · 蛋白质数据获取：蛋白质数据银行（PDB）

**注意：** 本教学教程是 TeachOpenCADD 的一部分，该平台旨在教授领域特定技能并为研究项目提供管道模板。

作者：

- Anja Georgi, CADD seminar, 2017, Charité/FU Berlin
- Majid Vafadar, CADD seminar, 2018, Charité/FU Berlin
- Jaime Rodríguez-Guerra, [Volkamer lab, Charité](https://volkamerlab.org/)
- Dominique Sydow, [Volkamer lab, Charité](https://volkamerlab.org/)

__教学教程 T008__：本教程是 TeachOpenCADD 管道的一部分，该管道在第一篇 TeachOpenCADD 发表文章中描述（[_J. Cheminform._ (2019), **11**, 1-7](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-019-0351-x)），包括教学教程 T001-T010。

在本教学教程中，我们为下一个教学教程做基础工作，在下一个教程中我们将为 BRCA1 生成基于配体的集成药效团。因此，我们将：
(i) 从 PDB 数据库中获取满足特定条件的 BRCA1 的所有 PDB ID（例如，高分辨率的配体结合结构），
(ii) 获取具有最佳结构质量的蛋白质-配体结构，
(iii) 对齐所有结构，以及
(iv) 提取并保存配体，用于下一个教学教程。

### 理论内容

* 蛋白质数据银行（PDB）
* 使用 Python 包 `biotite` 和 `pypdb` 查询 PDB

### 实践内容

* 选择查询蛋白质
* 获取查询蛋白质的 PDB 条目数量
* 查找满足特定条件的 PDB 条目
* 选择分辨率最高的 PDB 条目
* 从顶级结构中获取配体的元数据
* 绘制顶级配体分子
* 创建蛋白质-配体 ID 对
* 对齐 PDB 结构并提取配体

### 参考文献

* Protein Data Bank 
([PDB 网站](http://www.rcsb.org/))
* `pypdb` Python 包 
([_Bioinformatics_ (2016), **1**, 159-60](https://academic.oup.com/bioinformatics/article-lookup/doi/10.1093/bioinformatics/btv543); [文档](http://www.wgilpin.com/pypdb_docs/html/))
* `biotite` Python 包 ([_BMC Bioinformatics_ (2018), **19**](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2367-z); [文档](https://www.biotite-python.org/))
* 使用 Python 包 `opencadd` 进行分子叠加 ([仓库](https://github.com/volkamerlab/opencadd))

## 理论

### 蛋白质数据银行（PDB）

RCSB 蛋白质数据银行（PDB）是一个全面的结构生物学信息数据库，是结构生物学领域的关键资源，如结构基因组学和药物设计（[PDB 网站](http://www.rcsb.org/)）。

结构数据来源于结构测定方法，如 X 射线晶体学（最常用方法）、核磁共振（NMR）和冷冻电子显微镜（cryo-EM）。
对于每个条目，数据库包含（i）蛋白质、配体、辅因子、水分子和离子的原子 3D 坐标以及连接这些原子的化学键，以及（ii）结构数据的元信息，如 PDB ID、作者、提交日期、使用的结构测定方法和结构分辨率。
结构分辨率是收集数据质量的度量，单位是 Å（埃）；数值越低，结构质量越高。

PDB 网站提供蛋白质结构的 3D 可视化（如果可用，还包括配体相互作用）和结构质量指标，这可以在乳腺癌易感基因1（BRCA1）的示例 PDB 条目 [3UG5](https://www.rcsb.org/structure/3UG5) 中看到。

![protein ligand complex](images/protein-ligand-complex.png)

*图 1*：显示了乳腺癌易感基因1（BRCA1）的示例 PDB ID 3UG5 的蛋白质结构（灰色）与相互作用配体（绿色）。

### 使用 Python 包 `biotite` 和 `pypdb` 查询 PDB

PDB 数据库中的每个结构都链接到许多不同的字段来保存元信息。查看 [化学物质](https://search.rcsb.org/chemical-search-attributes.html)/[结构](https://search.rcsb.org/structure-search-attributes.html) 的可用字段完整列表和 RCSB 网站上支持的操作符。Python 包 `biotite` 提供了便捷的模块 `databases.rcsb`（参见[文档](https://www.biotite-python.org/apidoc/biotite.database.rcsb.html)），它允许我们查询这些字段中的一个（`FieldQuery`，参见[文档](https://www.biotite-python.org/apidoc/biotite.database.rcsb.FieldQuery.html#biotite.database.rcsb.FieldQuery)）或多个（`CompositeQuery`，参见[文档](https://www.biotite-python.org/apidoc/biotite.database.rcsb.CompositeQuery.html#biotite.database.rcsb.CompositeQuery)）来检索匹配我们条件的 PDB ID 计数（`count`）或列表（`search`）。

Python 包 `pypdb` 为 PDB 提供了一个接口，不仅可以查询 PDB ID，还可以下载相关的元数据和结构文件（[_Bioinformatics_ (2016), **1**, 159-60](https://academic.oup.com/bioinformatics/article-lookup/doi/10.1093/bioinformatics/btv543)，[文档](http://www.wgilpin.com/pypdb_docs/html/)）。查看介绍 `pypdb` API 的 [演示](https://github.com/williamgilpin/pypdb/blob/master/demos/demos.ipynb) 笔记本。

在本笔记本中，我们将使用这两个包：`biotite` 来快速过滤 PDB 中的许多结构，给定特定条件，`pypdb` 来下载特定感兴趣的 PDB 条目的元数据和结构文件。

## 实践

In [ ]:
import collections
import logging
import pathlib
import time
import warnings
import datetime

import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from tqdm.auto import tqdm
import redo
import requests_cache
import nglview
import pypdb
import biotite.database.rcsb as rcsb
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools

from opencadd.structure.superposition.api import align, METHODS
from opencadd.structure.core import Structure

# 禁用一些不需要的警告
logger = logging.getLogger("opencadd")
logger.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# 缓存请求 -- 这将加速对 PDB 的重复查询
requests_cache.install_cache("rcsb_pdb", backend="memory")

In [ ]:
# 定义路径
HERE = pathlib.Path(_dh[-1])
DATA = HERE / "data"

### 选择查询蛋白质

我们在本教学教程中使用 BRCA1 作为查询蛋白质。BRCA1 的 UniProt ID 是 `P38398`，将在下面用于查询 PDB 数据库。

In [ ]:
uniprot_id = "P38398"

### 获取查询蛋白质的 PDB 条目数量

BRCA1 在 PDB 中有多少可用结构（在本次笔记本最后运行时）？

In [ ]:
query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
today = datetime.datetime.now()
print(
    f"{today.year}-{today.month}-{today.day} 的结构数量：{rcsb.count(query_by_uniprot_id)}"
)

自 [PDB 于 1971 年建立](https://www.rcsb.org/pages/about-us/history) 以来，每年有多少可用结构？

In [ ]:
# 定义年份和在给定年份可用结构数量的列表
years = range(1971, datetime.datetime.now().year)
n_structures = []

for year in years:
    # 设置允许提交的最晚日期
    before_deposition_date = f"{year}-12-31T23:59:59Z"
    # 设置在给定日期之前提交的结构查询
    query_by_deposition_date = rcsb.FieldQuery(
        "rcsb_accession_info.deposit_date", less_or_equal=before_deposition_date
    )
    # 设置组合查询
    query = rcsb.CompositeQuery(
        [query_by_uniprot_id, query_by_deposition_date],
        "and",
    )
    # 计算匹配的结构并添加到列表
    n_structures.append(rcsb.count(query))
    # 短暂等待以不使 API 过载
    time.sleep(0.1)

绘制结果！

In [ ]:
plt.plot(years, n_structures)
plt.title("BRCA1 的 PDB 条目")
plt.xlabel("年份")
plt.ylabel("在给定年份可用的结构数量");

### 查找满足特定条件的 PDB 条目

我们将搜索描述 PDB 中满足以下条件的结构的 PDB ID：

- **UniProt ID P38398 的结构**：这是我们感兴趣的目标，BRCA1！
- **2020 年之前提交的结构**：这一步是为了 TeachOpenCADD 内部维护目的。我们将只考虑 2020 年之前提交的结构。这样，本次笔记本的结果将保持不变，使我们能够通过持续集成（CI）检查本次笔记本是否不会失效。
- **通过 X 射线晶体学解析的结构**：我们可以包括所有方法，但让我们查看一下 API 如何选择实验方法。
- **分辨率小于或等于 3.0 的结构**：分辨率值越低，结构质量越高，即分配的原子 3D 坐标正确的确定性越高。低于 3 Å 可以确定原子方向。因此，对于与基于结构药物设计相关的结构，这个阈值经常被使用。
- **只有一个链的结构**：我们这样做是为了让我们的生活更轻松。
- **具有分子量大于 100.0 Da 的配体的结构**：PDB 注释的配体可以是配体，也可以是溶剂和离子。为了只筛选配体结合的结构，我们只保留具有至少 100.0 Da 的注释配体的结构（许多溶剂和离子重量较少）。注意：这是一个简单但不是全面排除溶剂和离子的方法。

我们再次使用 `biotite` 包来基于以下组合查询查询 PDB ID：

| 字段 | 操作符 | 值 |
|-|-|-|
| `rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession` | `exact_match` | P38398 |
| `rcsb_accession_info.deposit_date` | `less` | 2020-01-01 |
| `exptl.method` | `exact_match` | X-RAY DIFFRACTION |
| `rcsb_entry_info.resolution_combined` | `less_or_equal` | $3.0$ |
| `rcsb_entry_info.deposited_polymer_entity_instance_count` | `equals` | $1$ |
| `chem_comp.formula_weight` | `greater` | $100.0$ |

我们定义我们的条件。

In [ ]:
uniprot_id = "P38398"

我们设置每个查询。

In [ ]:
query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_deposition_date = rcsb.FieldQuery(
    "rcsb_accession_info.deposit_date", less=before_deposition_date
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

我们单独执行每个查询并检查每个条件的匹配数量。

In [ ]:
print(f"具有 UniProt ID {uniprot_id} 的结构数量：{rcsb.count(query_by_uniprot_id)}")
time.sleep(0.1)  # 短暂等待以不使 API 过载
print(
    f"在 {before_deposition_date} 之前提交的结构数量：{rcsb.count(query_by_deposition_date)}"
)
time.sleep(0.1)
print(
    f"通过 {experimental_method} 解析的结构数量：{rcsb.count(query_by_experimental_method)}"
)
time.sleep(0.1)
print(
    f"分辨率小于或等于 {max_resolution} 的结构数量：{rcsb.count(query_by_resolution)}"
)
time.sleep(0.1)
print(f"只有 {n_chains} 个链的结构数量：{rcsb.count(query_by_polymer_count)}")
time.sleep(0.1)
print(
    f"具有大于或等于 {min_ligand_molecular_weight} Da 配体的结构数量：{rcsb.count(query_by_ligand_mw)}"
)

我们使用 `and` 操作符组合所有查询，以只匹配满足所有条件的 PDB ID。

In [ ]:
query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_deposition_date,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"匹配数量：{len(pdb_ids)}")
print("选定的 PDB ID：")
print(*pdb_ids)

### 选择分辨率最高的 PDB 条目

到目前为止，我们已经使用某些搜索条件来找到感兴趣的 PDB 条目。

目前，我们无法通过 `biotite` 直接访问结构的分辨率；使用 `biotite` 我们只能检查分辨率是否满足特定条件。相反，我们下载选定 PDB ID 的完整元数据。为此，我们使用 `pypdb` 包中的方法 `describe_pdb`。每个结构的元数据作为字典返回。

注意：我们这里只获取 PDB 结构的元信息，我们还没有获取结构（3D 坐标）。

> `redo.retriable` 行是一个_装饰器_。这包装了函数并提供额外的功能。在这种情况下，它将自动重试失败的查询（最多 10 次）。

In [ ]:
@redo.retriable(attempts=10, sleeptime=2)
def describe_one_pdb_id(pdb_id):
    """从 PDB 获取元信息。"""
    described = pypdb.describe_pdb(pdb_id)
    if described is None:
        print(f"! 获取 {pdb_id} 时出错，重试中...")
        raise ValueError(f"无法获取 PDB id {pdb_id}")
    return described

In [ ]:
pdbs_data = [describe_one_pdb_id(pdb_id) for pdb_id in tqdm(pdb_ids)]

让我们看一下前几个 PDB ID 的元数据（只有键，因为字典包含大量信息，我们不想在这里打印）。在 [PDB 结构和 PDBx/mmCIF 格式初学者指南](https://pdb101.rcsb.org/learn/guide-to-understanding-pdb-data/beginner%E2%80%99s-guide-to-pdb-structures-and-the-pdbx-mmcif-format) 中查找更多关于 PDB 元数据的信息。

In [ ]:
print("\n".join(pdbs_data[0].keys()))

让我们更仔细地看看两个对我们来说很重要的键：包含 PDB ID（`"id"`）的 `"entry"` 键和包含结构分辨率（`"resolution_combined"`）等的 `"rcsb_entry_info"` 键。

In [ ]:
pdbs_data[0]["entry"]

In [ ]:
pdbs_data[0]["rcsb_entry_info"]

现在我们将每个 PDB ID 的分辨率保存为 `pandas` DataFrame，按分辨率升序排序。

In [ ]:
resolution = pd.DataFrame(
    [
        [pdb_data["entry"]["id"], pdb_data["rcsb_entry_info"]["resolution_combined"][0]]
        for pdb_data in pdbs_data
    ],
    columns=["pdb_id", "resolution"],
).sort_values(by="resolution", ignore_index=True)
resolution

### 从顶级结构中获取配体的元数据

In [ ]:
top_num = 6  # 顶级结构的数量

在下一个教学教程中，我们将从分辨率最高的前 `top_num` 个结构中构建基于配体的集成药效团。

In [ ]:
selected_pdb_ids = resolution[:top_num]["pdb_id"].to_list()
print(f"选定的 PDB ID：{selected_pdb_ids}")

选定的高分辨率 PDB 条目可能包含针对不同结合位点的配体，例如变构配体和正构配体，这将阻碍基于配体的药效团生成。因此，我们将专注于以下 4 个结构，它们包含正构结合口袋中的配体。笔记本后面提供的代码可用于验证这一点。

In [ ]:
selected_pdb_ids = ["5UG9", "5HG8", "5UG8", "5UGC"]
print(f"选定的 PDB ID（固定集合）：{selected_pdb_ids}")

我们使用 `get_ligands` 获取前 `top_num` 个配体的 PDB 信息，存储为 *csv* 文件（每个配体作为字典）。

如果结构包含多个配体，我们选择最大的配体。注意：这是一种简单但不是全面的选择蛋白质结合位点中配体的方法。这种方法也可能选择与蛋白质结合的辅因子。因此，请在进一步使用之前目视检查自动选择的顶级配体。

In [ ]:
def get_ligands(pdb_id):
    """
    RCSB 尚未提供配体信息的新端点。作为
    变通方法，我们正从 ligand-expo.rcsb.org 获取额外信息，
    使用 HTML 解析。查看教学教程 T011 了解更多关于此技术的信息！
    """
    pdb_info = _fetch_pdb_nonpolymer_info(pdb_id)
    ligand_expo_ids = [
        nonpolymer_entities["pdbx_entity_nonpoly"]["comp_id"]
        for nonpolymer_entities in pdb_info["data"]["entry"]["nonpolymer_entities"]
    ]

    ligands = {}
    for ligand_expo_id in ligand_expo_ids:
        ligand_expo_info = _fetch_ligand_expo_info(ligand_expo_id)
        ligands[ligand_expo_id] = ligand_expo_info

    return ligands


def _fetch_pdb_nonpolymer_info(pdb_id):
    """
    从 rcsb.org 获取非聚合物数据。
    感谢 @BJWiley233 和 Rachel Green 提供此 GraphQL 解决方案。
    """
    query = (
        """{
          entry(entry_id: "%s") {
            nonpolymer_entities {
              pdbx_entity_nonpoly {
                comp_id
                name
                rcsb_prd_id
              }
            }
          }
        }"""
        % pdb_id
    )

    query_url = f"https://data.rcsb.org/graphql?query={query}"
    response = requests.get(query_url)
    response.raise_for_status()
    info = response.json()
    return info


def _fetch_ligand_expo_info(ligand_expo_id):
    """
    从 ligand-expo.rcsb.org 获取配体数据。
    """
    r = requests.get(f"http://ligand-expo.rcsb.org/reports/{ligand_expo_id[0]}/{ligand_expo_id}/")
    r.raise_for_status()
    html = BeautifulSoup(r.text)
    info = {}
    for table in html.find_all("table"):
        for row in table.find_all("tr"):
            cells = row.find_all("td")
            if len(cells) != 2:
                continue
            key, value = cells
            if key.string and key.string.strip():
                info[key.string.strip()] = "".join(value.find_all(string=True))

    # 后处理一些已知值
    info["Molecular weight"] = float(info["Molecular weight"].split()[0])
    info["Formal charge"] = int(info["Formal charge"])
    info["Atom count"] = int(info["Atom count"])
    info["Chiral atom count"] = int(info["Chiral atom count"])
    info["Bond count"] = int(info["Bond count"])
    info["Aromatic bond count"] = int(info["Aromatic bond count"])
    return info

In [ ]:
columns = [
    "@structureId",
    "@chemicalID",
    "@type",
    "@molecularWeight",
    "chemicalName",
    "formula",
    "InChI",
    "InChIKey",
    "smiles",
]
rows = []
for pdb_id in selected_pdb_ids:
    ligands = get_ligands(pdb_id)
    # 如果包含多个配体，取最大的（结果中的第一个）
    ligand_id, properties = max(ligands.items(), key=lambda kv: kv[1]["Molecular weight"])
    rows.append(
        [
            pdb_id,
            ligand_id,
            properties["Component type"],
            properties["Molecular weight"],
            properties["Name"],
            properties["Formula"],
            properties["InChI descriptor"],
            properties["InChIKey descriptor"],
            properties["Stereo SMILES (OpenEye)"],
        ]
    )

In [ ]:
# NBVAL_CHECK_OUTPUT
# 更改格式为 DataFrame
ligands = pd.DataFrame(rows, columns=columns)
ligands

In [ ]:
ligands.to_csv(DATA / "PDB_top_ligands.csv", header=True, index=False)

### 绘制顶级配体分子

In [ ]:
PandasTools.AddMoleculeColumnToFrame(ligands, "smiles")
Draw.MolsToGridImage(
    mols=list(ligands.ROMol),
    legends=list(ligands["@chemicalID"] + ", " + ligands["@structureId"]),
    molsPerRow=top_num,
)

### 创建蛋白质-配体 ID 对

In [ ]:
# NBVAL_CHECK_OUTPUT
pairs = collections.OrderedDict(zip(ligands["@structureId"], ligands["@chemicalID"]))
pairs

### 对齐 PDB 结构并提取配体

由于我们要在下一个教学教程中构建基于配体的集成药效团，因此有必要在 3D 中将所有结构相互对齐。

我们将使用 Python 包 `opencadd`（[仓库](https://github.com/volkamerlab/opencadd)），其中包括一个 3D 叠加子包来指导蛋白质的结构对齐。该方法基于提供的匹配残基的序列比对引导的叠加。包中有其他方法，但这个简单的方法对于手头的任务来说就足够了。

#### 获取 PDB 结构文件

我们现在使用 `opencadd.structure.superposition` 从 PDB 获取 PDB 结构文件，即蛋白质、配体（以及其他原子或分子实体，如辅因子、水分子和离子，如果可用）的 3D 坐标。

可用的文件格式是 *pdb* 和 *cif*，它们存储蛋白质（和配体、辅因子、水分子和离子）的原子 3D 坐标以及原子间化学键的信息。这里，我们使用 *pdb* 文件。

In [ ]:
# 下载 PDB 结构
structures = [Structure.from_pdbid(pdb_id) for pdb_id in pairs]
structures

#### 提取蛋白质和配体

从结构中提取蛋白质和配体，以去除溶剂和其他结晶学产物。

In [ ]:
complexes = [
    Structure.from_atomgroup(structure.select_atoms(f"protein or resname {ligand}"))
    for structure, ligand in zip(structures, pairs.values())
]
complexes

In [ ]:
# 将复合物写入文件
for complex_, pdb_id in zip(complexes, pairs.keys()):
    complex_.write(DATA / f"{pdb_id}.pdb")

#### 对齐蛋白质

对齐复合物（基于蛋白质原子）。

In [ ]:
results = align(complexes, method=METHODS["mda"])

`nglview` 可用于在 Jupyter 笔记本中可视化分子数据。在下一个单元格中，我们将可视化对齐的蛋白质-配体复合物。

In [ ]:
view = nglview.NGLWidget()
for complex_ in complexes:
    view.add_component(complex_.atoms)
view

In [ ]:
view.render_image(trim=True, factor=2, transparent=True);

In [ ]:
view._display_image()

#### 提取配体

In [ ]:
ligands = [
    Structure.from_atomgroup(complex_.select_atoms(f"resname {ligand}"))
    for complex_, ligand in zip(complexes, pairs.values())
]
ligands

In [ ]:
for ligand, pdb_id in zip(ligands, pairs.keys()):
    ligand.write(DATA / f"{pdb_id}_lig.pdb")

我们检查所有配体 *pdb* 文件的存在。

In [ ]:
ligand_files = []
for file in DATA.glob("*_lig.pdb"):
    ligand_files.append(file.name)
ligand_files

我们也可以使用 `nglview` 来单独显示共结晶配体。正如我们所看到的，所选的复合物包含占据相同结合口袋的配体，因此可以在下一个教学教程中用于基于配体的药效团生成。

In [ ]:
view = nglview.NGLWidget()
for component_id, ligand in enumerate(ligands):
    view.add_component(ligand.atoms)
    view.remove_ball_and_stick(component=component_id)
    view.add_licorice(component=component_id)
view

In [ ]:
view.render_image(trim=True, factor=2, transparent=True);

In [ ]:
view._display_image()

## 讨论

在本教学教程中，我们学习了如何从 PDB 中检索蛋白质和配体的元信息和结构信息。我们只保留了 BRCA1 的 X 射线结构，并通过分辨率和配体可用性过滤了我们的数据。最终，我们的目标是获得一组对齐的配体，用于下一个教学教程中生成基于配体的集成药效团。

为了丰富药效团建模的配体信息，建议不仅按 PDB 结构分辨率过滤，还要检查配体多样性（参见 **教学教程 T005** 关于按相似性进行分子聚类）和检查配体活性（即只包括强效配体）。

在技术方面：我们看到，存储在 PDB 中的不同信息片段（结构元数据、坐标、配体元数据）目前无法从单个 Python 包访问，但将 `biotite` 和 `pypdb` 结合使用——以及一些网页抓取来提取配体信息——我们能够收集我们需要的所有信息。鉴于 PDB 最近在（2020 年 11 月）[完全切换到新的 API](https://www4.rcsb.org/news?year=2020&article=5eb18ccfd62245129947212a&feature=true)，相关的开源 Python 包在未来可能会在有时间追赶之后提供更多功能。

## 测验

1. 总结蛋白质数据银行包含的数据类型。
2. 解释结构分辨率的含义以及我们在此教学教程中如何以及为什么过滤它。
3. 解释结构对齐的含义并讨论本教学教程中执行的对齐。